In [1]:
import alphaspace,mdtraj,os
import numpy as np

In [2]:
print(os.getcwd())

/Users/haotian/Dropbox/pycharm_project/AlphaSpace


In [85]:
al = mdtraj.load('examples/match/AL.pdb')

universe1 = alphaspace.AS_Universe(mdtraj.load('examples/match/AL.pdb'),guess_receptor_binder=True,guess_by_order=True,tag='AL')
universe2 = alphaspace.AS_Universe(mdtraj.load('examples/match/AB.pdb'),guess_receptor_binder=True,guess_by_order=True,tag="AB")

In [87]:
for molecule in al.topology.find_molecules():
    print(len(molecule))

3388
302
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [84]:
# Creating overlapping atom set and slice the trajectory

atom_list1 = set([(atom.name,atom.residue.index) for atom in list(universe1.receptor.atoms)])
atom_list2 = set([(atom.name,atom.residue.index) for atom in list(universe2.receptor.atoms)])

shared_atom_list = atom_list2.intersection(atom_list1)

for universe in universe1,universe2:
    kept_atom_indice = []
    for atom in universe.receptor.atoms:
        if (atom.name,atom.residue.index) in shared_atom_list:
            kept_atom_indice.append(atom.index)
    universe.receptor.traj.atom_slice(inplace=True,atom_indices=kept_atom_indice)   
    
for residue in universe1.binder.residues:
    print(residue.index, residue.name)
print()
for residue in universe2.binder.residues:
    print(residue.index, residue.name)

428 GLY
429 PRO
430 VAL
431 LEU
432 THR
433 PHE
434 GLN
435 SER
436 GLU
437 LYS
438 MET
439 LYS
440 GLY
441 MET
442 LYS
443 GLU
444 LEU
445 LEU
446 VAL
447 ALA
448 THR
449 LYS
450 ILE
451 ASN
452 SER
453 SER
454 ALA
455 ILE
456 LYS
457 LEU
458 GLN
459 LEU
460 THR
461 ALA
462 GLN
463 SER
464 GLN
465 VAL
466 GLN
467 MET
468 LYS

467 SER
468 ALA
469 ARG
470 TRP
471 ARG
472 PHE
473 PRO
474 ALA
475 ARG
476 PRO
477 GLY
478 THR


In [65]:
universe1.config.screen_by_face = universe2.config.screen_by_face = False
universe1.config.screen_by_lig_cntct = universe2.config.screen_by_lig_cntct = True

In [66]:
universe1.run_alphaspace()
universe2.run_alphaspace()

0 snapshot processed


0 snapshot processed


In [81]:
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import squareform

pocket_list = (list(universe1.pockets(snapshot_idx=0,active_only=False))+list(universe2.pockets(snapshot_idx=0,active_only=False)))

pocket_diff_matrix = np.ones((len(pocket_list),len(pocket_list))) * 10

for i, pocket1 in enumerate(pocket_list):
    # print(pocket1.lining_atoms_idx)
    for j, pocket2 in enumerate(pocket_list):
        if pocket1.parent_structure.universe.index != pocket2.parent_structure.universe.index and i > j:
            lining_atom1 = pocket1.lining_atoms_idx
            lining_atom2 = pocket2.lining_atoms_idx
            
            pocket_diff_matrix[i][j] = pocket_diff_matrix[j][i] = 1- float(len(np.intersect1d(lining_atom1,lining_atom2,assume_unique=True)))/len(np.unique(np.concatenate([lining_atom1,lining_atom2])))
            # print(pocket_diff_matrix[i][j])
        if i == j :
            pocket_diff_matrix[i][i] = 0

# for pocket in universe2.pockets():
#     for i in pocket.lining_atoms_idx:
#         print(i, universe2.receptor.atom(i).name)
#     print("pocket")
        
zmat = linkage(squareform(pocket_diff_matrix),method='average')
pocket_cluster_indices = fcluster(zmat, 0.75,criterion='distance') - 1

In [82]:
matched_pockets = [[] for _ in range(max(pocket_cluster_indices)+1)]
for pocket_index, pocket_cluster_index in enumerate(pocket_cluster_indices):
    matched_pockets[pocket_cluster_index].append(pocket_list[pocket_index])

for i in matched_pockets:
    print(i)

[<Pocket 51 in snapshot 0 with space 107 and 0% occupied>, <Pocket 61 in snapshot 0 with space 91 and 0% occupied>]
[<Pocket 221 in snapshot 0 with space 8 and 0% occupied>, <Pocket 248 in snapshot 0 with space 4 and 0% occupied>]
[<Pocket 237 in snapshot 0 with space 5 and 0% occupied>, <Pocket 237 in snapshot 0 with space 5 and 0% occupied>]
[<Pocket 96 in snapshot 0 with space 60 and 0% occupied>, <Pocket 179 in snapshot 0 with space 18 and 0% occupied>]
[<Pocket 198 in snapshot 0 with space 13 and 0% occupied>, <Pocket 54 in snapshot 0 with space 98 and 0% occupied>]
[<Pocket 116 in snapshot 0 with space 50 and 0% occupied>, <Pocket 126 in snapshot 0 with space 40 and 0% occupied>]
[<Pocket 185 in snapshot 0 with space 17 and 0% occupied>]
[<Pocket 172 in snapshot 0 with space 20 and 0% occupied>]
[<Pocket 195 in snapshot 0 with space 15 and 0% occupied>, <Pocket 190 in snapshot 0 with space 16 and 0% occupied>]
[<Pocket 140 in snapshot 0 with space 39 and 0% occupied>, <Pocket 114